## Test Bert

In [26]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [29]:
max_text_len=900
max_summary_len=100

In [27]:
post_pre = pd.read_csv ('..\Dataset\Indonesian News Dataset Preprocessed.csv')

In [21]:
#Add sostok and eostok at 
post_pre['summary'] = post_pre['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

post_pre['text'] = post_pre['text'].astype(str)
post_pre['summary'] = post_pre['summary'].astype(str)

In [22]:
x_tr,x_val,y_tr,y_val=train_test_split(np.array(post_pre['text']),np.array(post_pre['summary']),test_size=0.1,random_state=0,shuffle=True)

In [23]:
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

In [24]:
thresh=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 61.07469711496557
Total Coverage of rare words: 1.1015455904901386


In [30]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences (i.e one-hot encodeing all the words)
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

print("Size of vocabulary in X = {}".format(x_voc))

Size of vocabulary in X = 44660


In [31]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

In [32]:
thresh=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 58.33826851790924
Total Coverage of rare words: 2.078358999120164


In [33]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences (i.e one hot encode the text in Y)
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1
print("Size of vocabulary in Y = {}".format(y_voc))

Size of vocabulary in Y = 18995


In [34]:
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

In [35]:
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

In [1]:
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization")

pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

c:\Users\Sanja\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sanja\.cache\huggingface\hub\models--cahya--bert2bert-indonesian-summarization. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [44]:
# 
ARTICLE_TO_SUMMARIZE = "menteri bumn erick thohir penjabat gubernur dki jakarta heru budi hartono dan direktur utama pertamina nicke widyawati siang ini langsung menggelar rapat di depo plumpang jakarta utara setelah diperintah presiden joko widodo atau jokowi jokowi memerintahkan agar ada solusi dalam satu dua hari ini usai kebakaran di depo pertamina apakah fasilitas penyimpanan bensin ini dipindah atau warga di sekitar di relokasi ini mau rapat di sana ini rapat langsung kata erick usai menemani jokowi yang mengunjungi para pengungsi kebakaran depo pertamina di rptra rasela koja jakarta utara ahad maret 2023.jokowi datang bersama erick heru yang juga kepala sekretariat presiden dan nicke nicke juga membenarkan rapat digelar di plumpang bersama erick dan heru iya kata dia di lokasi.di lokasi ini jokowi telah memerintahkan erick heru dan pertamina mencari solusi usai kejadian kebakaran di depo pertamina plumpang jakarta utara jokowi memberi dua opsi depo pertamina dipindahkan atau warga yang tinggal direlokasi ini akan segera diputuskan sehari dua hari ini oleh pertamina gubernur dki sehingga solusinya menjadi jelas kata jokowi.ada dua opsijokowi belum bisa memastikan apakah nanti warga masih bisa tinggal di lokasi jika nanti yang akhirnya depo pertamina yang dipindah ini yang baru nanti dibicarakan ada pilihan-pilihan apakah deponya yang digeser apakah masyarakatnya yang digeser kalau digeser tanahnya di mana tapi harus segera ditemukan solusinya kata dia.jokowi menyebut depo pertamina plumpang ini merupakan zona bahaya tidak bisa lagi ditinggali tetapi harus ada solusinya bisa saja plumpang-nya digeser ke reklamasi atau penduduknya yang digeser ke relokasi mantan gubernur dki jakarta 2012-2014.jokowi tidak merinci lebih lanjut reklamasi yang dia maksud menurut dia solusi akan dibicarakan oleh pt pertamina dan heru tapi semuanya memang harus zona-zona berbahaya ini tidak hanya di sini saja harus diaudit harus dievaluasi semuanya karena menyangkut nyawa tadi saya sudah perintahkan semuanya ujarnya.jokowi menyebut seharusnya depo pertamina ini harusnya menjadi zona air seperti misalnya dibuat sungai yang memisahkan depo dan pemukiman tujuannya untuk melindungi objek vital yang ada di dalam depo karena barang-barang didalamnya barang-barang yang sangat bahaya untuk berdekatan dengan masyarakat apalagi dengan pemukiman penduduk kata dia.wapres usul depo dipindah ke pelabuhankebakaran terjadi jumat malam maret 2023 yang menewaskan 17 orang dan 600 lebih mengungsi sebelum jokowi wakil presiden ma ruf amin sudah lebih dulu mengunjungi pengungsi di plumpang.ma ruf mengusulkan agar depo pertamina di plumpang dipindah menjauhi permukiman penduduk pascaterbakar pada jumat malam kemarin kebakaran depo menjalar ke pemukiman warga hingga mengakibatkan jatuhnya korban jiwa saya berharap supaya depo ini lebih aman itu bisa direlokasi di pelabuhan di daerah pelindo saya kira begitu ujar ma ruf amin di kawasan plumpang jakarta utara sabtu maret 2023."

# generate summary
input_ids = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors='pt')
summary_ids = model.generate(input_ids,
            min_length=20,
            max_length=80, 
            num_beams=10,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.8,
            top_k = 50,
            top_p = 0.95)

summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

presiden joko widodo ( sby ) memerintahkan agar ada solusi dalam satu dua hari ini usai kebakaran di depo pertamina plumpang jakarta utara.


## Test Spacy

In [4]:
import spacy
from spacy.lang.id import Indonesian

# Load Indonesian spaCy model
nlp = Indonesian()

# Define a text to tokenize
text = "proyek tersebut dilakukan di kota jakarta oleh perusahaan yang berbasis di singapura"

# Process the text with spaCy
processed_text = nlp(text)

# Print tokenized text
for token in processed_text:
    print(token.text)

proyek
tersebut
dilakukan
di
kota
jakarta
oleh
perusahaan
yang
berbasis
di
singapura


## Test Summarizer with Transformer|

In [29]:
from transformers import BertTokenizer, EncoderDecoderModel

In [30]:
tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization")

In [31]:
# Your text to summarize
text = """Perang Dunia II (PD II) adalah konflik global antara Sekutu dan Poros yang terjadi dari 1939 hingga 1945. Dipicu oleh ambisi ekspansionis Jerman Nazi di Eropa dan agresi Jepang di Asia, PD II mengakibatkan perubahan besar dalam politik, ekonomi, dan sosial dunia.

Peristiwa penting dalam PD II meliputi invasi Jerman ke Polandia pada 1939 yang memicu deklarasi perang oleh Inggris dan Prancis, serta serangan Jepang ke Pearl Harbor yang memaksa Amerika Serikat masuk dalam perang.

Sekutu, yang terdiri dari Amerika Serikat, Inggris, dan Uni Soviet, berhasil menghadapi Poros yang dipimpin oleh Jerman, Italia, dan Jepang. Konflik ini berakhir pada 1945 setelah bom atom dijatuhkan di Hiroshima dan Nagasaki, yang memaksa Jepang menyerah."""

In [32]:
# generate summary
input_ids = tokenizer.encode(text, return_tensors='pt')
summary_ids = model.generate(input_ids,
            min_length=20,
            max_length=80, 
            num_beams=10,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.8,
            top_k = 50,
            top_p = 0.95)

summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

konflik global antara sekutu dan poros yang terjadi dari perang dunia ii mengakibatkan perubahan besar dalam politik, ekonomi, dan sosial dunia.
